In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime, timedelta

from deps.data_prep import get_seconds_data
from deps.visualization import plot_timeline
from deps.runner import run_simulation

alt.data_transformers.disable_max_rows()
alt.renderers.enable('html')

np.set_printoptions(linewidth=300)

In [ ]:
weather_df = get_seconds_data(date_from="2020-07-01", date_to="2020-07-02")

In [ ]:
width, height = 300, 150
plot_timeline(weather_df, "temp", width=width, height=height) | \
plot_timeline(weather_df, "solarradiation", width=width, height=height) | \
plot_timeline(weather_df, "humidity", width=width, height=height)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>